In [18]:
import torch
print(torch.__version__)

1.12.1


In [24]:
# create sample node feature
X = torch.Tensor(torch.randn(6,3)) # torch.randn(#_of_node_feature, feature_dim)

In [25]:
# define sample adjacency matrix
adj = torch.Tensor([[0,1,0,0,1,0],
                    [1,0,1,0,1,0],
                    [0,1,0,1,0,0],
                    [0,0,1,0,1,1],
                    [1,1,0,1,0,0],
                    [0,0,0,1,0,0]])

In [26]:
print(adj)
print(adj.shape)

tensor([[0., 1., 0., 0., 1., 0.],
        [1., 0., 1., 0., 1., 0.],
        [0., 1., 0., 1., 0., 0.],
        [0., 0., 1., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.]])
torch.Size([6, 6])


In [27]:
# need self loops to use self informations
tilde = adj + torch.eye(adj.shape[0])
print(tilde)

tensor([[1., 1., 0., 0., 1., 0.],
        [1., 1., 1., 0., 1., 0.],
        [0., 1., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0.],
        [0., 0., 0., 1., 0., 1.]])


In [28]:
# to sparse matrix
sps = tilde.to_sparse()
print(sps.indices())
print(sps.values())
print(sps.size())

tensor([[0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5],
        [0, 1, 4, 0, 1, 2, 4, 1, 2, 3, 2, 3, 4, 5, 0, 1, 3, 4, 3, 5]])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])
torch.Size([6, 6])


$$
Z = \tilde{D}^{-{1\over2}} \tilde{A} \tilde{D}^{-{1\over2}} X\theta
$$

위 식에서 normalize 부분($\tilde{D}^{-{1\over2}}$)은 잠시 제외하고 계산을 해보자. ($X\theta$도 여기서는 초기화된 행렬 X로 가정)

In [29]:
print(X)
print(tilde)

tensor([[ 1.7163,  0.9421,  1.5012],
        [-0.7211, -1.1423,  0.0231],
        [-0.0166, -1.2985, -1.9682],
        [ 0.7414,  0.1619,  0.4191],
        [-0.5905, -1.0229, -1.6215],
        [-0.0281, -1.4829,  0.7234]])
tensor([[1., 1., 0., 0., 1., 0.],
        [1., 1., 1., 0., 1., 0.],
        [0., 1., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0.],
        [0., 0., 0., 1., 0., 1.]])


In [31]:
Z = torch.matmul(tilde, X)
print(Z)

tensor([[ 0.4046, -1.2230, -0.0972],
        [ 0.3881, -2.5215, -2.0653],
        [ 0.0038, -2.2788, -1.5259],
        [ 0.1062, -3.6423, -2.4472],
        [ 1.1461, -1.0611,  0.3220],
        [ 0.7133, -1.3209,  1.1425]])
